In [1]:
import galsim
import numpy as np
import matplotlib.pyplot as plt
from torch.distributions import Uniform, Beta, Normal
import pyccl
import torch

In [3]:
dist = Normal(torch.tensor([0.8159]), torch.tensor([0.001]))
sigma_8 = dist.sample().item()
print(sigma_8)

0.8168158531188965


In [11]:
cosmology = pyccl.cosmology.Cosmology(sigma8=sigma_8,
                                      Omega_c=2.6,
                                      Omega_b=0.0486,
                                      h=67.7,
                                      n_s=1,
)

In [13]:
cosmology.compute_linear_power()
power_spectrum = cosmology.get_linear_power()

 integral, error, tol:   57.829814798906504       -3.5592934681583088E-005   1.0000000000000001E-007


In [31]:
power_spectrum_vals_pk = power_spectrum.get_spline_arrays()[2]
power_spectrum_vals_k = power_spectrum.get_spline_arrays()[1]
# print(len(power_spectrum_vals))
# print(len(power_spectrum_vals[0]))
with open('./sample_ps_vals.txt', 'w') as f:
    np.savetxt(f, np.concat(power_spectrum_vals_k[0], power_spectrum_vals_pk[0], axis=1))

AttributeError: module 'numpy' has no attribute 'cat'

In [24]:
def get_ps():
    return power_spectrum_vals

In [ ]:
for arg in args:
    val = calculateVal(arg)
    vals.append(val)
table = galsim.LookupTable(x=args,f=vals)

In [25]:
galsim_ps = galsim.PowerSpectrum(get_ps, units=galsim.radians)

TypeError: get_ps() takes 0 positional arguments but 1 was given